In [1]:
%matplotlib inline

# This file contains all the main external libs fastai needs
from fastai.imports import *

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


the fastai notebooks imports all the functions into the global namespace, which I find confusing since I don't know what methods are fastai and what are coming from other libraries. 

Ideally I'd like to be able to just call `import fastai as fastai` rather than having multiple import statements. Anyways, using the defaults for now:

In [2]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.is_available(), torch.backends.cudnn.enabled

(True, True)

# Data

I'm going to use a pretrained NN to solve [Kaggle's digit recognizer competition](https://www.kaggle.com/c/digit-recognizer/data), cause why not. I used https://github.com/floydwch/kaggle-cli to download the dataset, and look at the [fastai dataset.py file](https://github.com/fastai/fastai/blob/master/fastai/dataset.py) to figure out how to use it.

In [4]:
PATH = os.getcwd() + '/data/mnist/'
print(PATH)
os.listdir(PATH)

/home/ubuntu/fastai_deeplearn/data/mnist/


['test.csv',
 'tmp',
 'sample_submission.csv',
 'train.csv',
 '.kaggle-cli',
 'models']

Now to take a look at the data:

In [5]:
train = pd.read_csv(PATH + "train.csv")
print(train.shape)
train.head(3)

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test = pd.read_csv(PATH+"test.csv")
print(test.shape)
test.head(3)

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We need the data in a form the fast.ai library can ingest, so in this case I'll pass in numpy arrays after splitting the training set into a train and validation split.

pytorch/fastai want image data to be in the same: `num_images, channels, height, width`, so I reshape the numpy arrays into 3 channel:

In [9]:
# grabbing the images from the datafram
train_images = train.iloc[:,1:].values.astype('float64')
# stacking them and reshaping
train_images_3D = (np.stack((train_images,)*3, -1)).reshape(-1, 3, 28, 28)
train_images_3D.shape

(42000, 3, 28, 28)

Now to split into train and validation images:

In [11]:
msk = np.random.rand(len(train)) < 0.85

x_train = train_images_3D[msk]
x_val = train_images_3D[~msk]

y_train = train.iloc[:,0][msk].values.astype('int32') # only labels i.e targets digits
y_val = train.iloc[:,0][~msk].values.astype('int32') # only labels i.e targets digits

x_train.shape, y_train.shape, x_val.shape, y_val.shape

((35758, 3, 28, 28), (35758,), (6242, 3, 28, 28), (6242,))

Finally, we can make the fast.ai data object.

In [24]:
arch = resnet34
data = ImageClassifierData.from_arrays(PATH, trn=(x_train,y_train), 
                                       val=(x_val,y_val), 
                                       tfms=tfms_from_model(arch, 54))

# fitting the model to the data

In [25]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.924436   0.71099    0.768045  
    1      0.833427   0.645011   0.788115                    



[0.6450113, 0.788115245955331]